In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Creating a synthetic trace with ghost, air gun bubbles and noise

In [ ]:
# Ricker Wavelet
nt = 1024
t=np.linspace(0,1.0,nt)
dt=t[2]-t[1]
f0=20 
t0=0.1
arg = np.pi*np.pi*f0*f0*(t-t0)**2
wav = (1-2*arg)*np.exp(-arg)

freqs = np.fft.fftfreq(nt)/dt;

#Adding ghost
zs = 10.
vw = 1500.
delay = 2*zs/vw
r0 = -1.
GhostFilter = 1.  + r0*np.exp(-1j*2*np.pi*freqs*delay)
wav = np.real(np.fft.ifft(np.fft.fft(wav)*GhostFilter))

# Adding monochromatic noise with freqs[100] Hz
noise = np.zeros([nt])
noise[150] = 50.

# Adding 5 Bubbles with bubble time period = 0.15 s and decaying amplitude
BubbleFilter = (1. + 0.5*np.exp(-1j*2*np.pi*freqs*0.15) + 
                                           0.25*np.exp(-1j*2*np.pi*freqs*0.3) + 
                                           0.125*np.exp(-1j*2*np.pi*freqs*0.45) + 
                                           0.0625*np.exp(-1j*2*np.pi*freqs*0.6) +
                                           0.03125*np.exp(-1j*2*np.pi*freqs*0.75))
trace = np.real(np.fft.ifft(noise + np.fft.fft(wav)*BubbleFilter))

# Ploting wavelet
plt.plot(t,trace)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid()



## Deghosting using division in the Frequency domain
## $D(f) = \frac{X(f)\overline{G(f)}}{G(f)\overline{G(f)} + \varepsilon}$

In [ ]:
epsilon = 1e-3
TRACE_FFT = np.fft.fft(trace)
trace_deghosted = np.real(np.fft.ifft(TRACE_FFT*np.conj(GhostFilter)/(GhostFilter*np.conj(GhostFilter) + epsilon)))

# Ploting data
plt.plot(t,trace_deghosted)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid()
                                                

## Deghosting using division in the Frequency domain (Phase only)
$\phi(f) = \tan\left[\frac{imag\{X(f)\}}{real\{X(f)\}}\right]^{-1}$ 

Full data:
$X(f) = A(f)\exp\left[-i\phi(f)\right]$

Phase only:
$G(f) = \exp\left[-i\phi(f)\right]$

In [ ]:
TRACE_FFT = np.fft.fft(trace)
Phase = np.arctan2(np.imag(GhostFilter),np.real(GhostFilter))
Ghost_Phase_only = np.exp(1j*Phase)
trace_deghosted2 = np.real(np.fft.ifft(TRACE_FFT*np.conj(Ghost_Phase_only)/(Ghost_Phase_only*np.conj(Ghost_Phase_only) + epsilon)))

# Ploting wavelet
plt.plot(t,trace_deghosted2)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid()
                

## Removing Bubble Filter in frequency domain


In [ ]:
TRACE_FFT = np.fft.fft(trace)
trace_debubble = np.real(np.fft.ifft(TRACE_FFT*np.conj(BubbleFilter)/(BubbleFilter*np.conj(BubbleFilter) + epsilon)))

# Ploting wavelet
plt.plot(t,trace_debubble)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid()



# Using a Wiener filter

In [ ]:
# Desired wavelet
desired = np.zeros([nt])
desired = trace_debubble

# Autocorrelation function 
A = np.convolve(trace, trace[::-1])
A = A[1023:-1]
norm = np.max(np.abs(A))
A = A/norm

# Correlation function (right hand side of system)
C = np.convolve(desired, trace[::-1])
C = C[1023:-1]
C = C/norm

# Setting up system
nf = 256 # Filter length
epsilon = 1e-3 # Damping (expected noise autocorrelation value)
M = np.zeros([nf,nf])
for i in range(0,nf):
    for j in range(0,nf):
        M[i,j] = A[np.abs(i-j)]
        if(i==j):
            M[i,j] = M[i,j] + epsilon

rhs = np.zeros([nf])
rhs[0:nf] = C[0:nf]

# Solving system
f = np.linalg.solve(M,rhs)

# Checking that solution is valid (in which case we get True)
np.allclose(np.dot(M, f), rhs)

In [ ]:
# Convolving trace with filter
tracefilt = np.convolve(trace, f)
tracefilt = tracefilt[0:nt]
plt.plot(t,tracefilt)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid()

## Spiking deconvolution

In [ ]:
# Autocorrelation function
A = np.convolve(trace[::-1], trace)
A = A[1023:-1]
norm = np.max(np.abs(A))
A = A/norm

# Setting up system
nf = 256
epsilon = 1
M = np.zeros([nf,nf])
for i in range(0,nf):
    for j in range(0,nf):
        M[i,j] = A[np.abs(i-j)]
        if(i==j):
            M[i,j] = M[i,j] + epsilon

rhs = np.zeros([nf])
rhs[0] = 1

# Solving system
f = np.linalg.solve(M,rhs)
np.allclose(np.dot(M, f), rhs)



In [ ]:
# Convolving trace with filter
tracefilt = np.convolve(trace, f)
tracefilt = tracefilt[0:nt]
plt.plot(t,tracefilt)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid()

## Predictive deconvolution

In [ ]:
# Autocorrelation function 
A = np.convolve(trace, trace[::-1])
A = A[1023:-1]
norm = np.max(np.abs(A))
A = A/norm

# Correlation function (right hand side of system)
delay = 0.10
idelay = int(np.floor((delay)/dt))
prelayed = np.zeros([nt]) # advanced trace
prelayed[0:nt-idelay] = trace[idelay:nt]
plt.plot(t,trace)
plt.plot(t,prelayed)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid()

C = np.convolve(prelayed, trace[::-1])
C = C[1023:-1]
C = C/norm

# Setting up system
nf = 256 # Filter length
epsilon = 1e-3 # Damping (expected noise autocorrelation value)
M = np.zeros([nf,nf])
for i in range(0,nf):
    for j in range(0,nf):
        M[i,j] = A[np.abs(i-j)]
        if(i==j):
            M[i,j] = M[i,j] + epsilon

rhs = np.zeros([nf])
rhs[0:nf] = C[0:nf]

# Solving system
f = np.linalg.solve(M,rhs)
np.allclose(np.dot(M, f), rhs)

In [ ]:
tracefilt = np.convolve(trace, f)
tracefilt = tracefilt[0:nt]
pred = np.zeros([nt])
pred[idelay:nt] = tracefilt[0:nt-idelay]

# Ploting trace and prediction 
plt.plot(t,trace)
plt.plot(t,pred)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid()

In [ ]:
# Ploting trace difference (prediction error)
plt.plot(t,(trace-pred))
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid()